In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

num_lda_topics = 8

In [35]:
df = pd.read_csv('../data/preprocessed_data.csv')

In [36]:
# group df by twitter user id   
user_ids = df['twitter user id'].unique()

In [37]:
# create dataframe from user_ids with their class label
df_user_ids = pd.DataFrame(user_ids, columns=['twitter user id'])
df_user_ids['class'] = df_user_ids['twitter user id'].apply(lambda x: df[df['twitter user id'] == x]['class'].unique()[0])
df_user_ids.shape  


(151, 2)

In [38]:
# create dataframe for each class
df_user_ids_0 = df_user_ids[df_user_ids['class'] == 'no influencer']
df_user_ids_1 = df_user_ids[df_user_ids['class'] == 'nano']
df_user_ids_2 = df_user_ids[df_user_ids['class'] == 'micro']
df_user_ids_3 = df_user_ids[df_user_ids['class'] == 'macro']
df_user_ids_4 = df_user_ids[df_user_ids['class'] == 'mega']

In [39]:
# create test , train and validate for each class dataframe
df_user_ids_0_train = df_user_ids_0.sample(frac=0.8, random_state=0)
df_user_ids_0_test = df_user_ids_0.drop(df_user_ids_0_train.index)
df_user_ids_0_validate = df_user_ids_0_test.sample(frac=0.5, random_state=0)
df_user_ids_0_test = df_user_ids_0_test.drop(df_user_ids_0_validate.index)

In [40]:
df_user_ids_1_train = df_user_ids_1.sample(frac=0.8, random_state=0)
df_user_ids_1_test = df_user_ids_1.drop(df_user_ids_1_train.index)
df_user_ids_1_validate = df_user_ids_1_test.sample(frac=0.5, random_state=0)
df_user_ids_1_test = df_user_ids_1_test.drop(df_user_ids_1_validate.index)
df_user_ids_2_train = df_user_ids_2.sample(frac=0.8, random_state=0)
df_user_ids_2_test = df_user_ids_2.drop(df_user_ids_2_train.index)
df_user_ids_2_validate = df_user_ids_2_test.sample(frac=0.5, random_state=0)
df_user_ids_2_test = df_user_ids_2_test.drop(df_user_ids_2_validate.index)
df_user_ids_3_train = df_user_ids_3.sample(frac=0.8, random_state=0)
df_user_ids_3_test = df_user_ids_3.drop(df_user_ids_3_train.index)
df_user_ids_3_validate = df_user_ids_3_test.sample(frac=0.5, random_state=0)
df_user_ids_3_test = df_user_ids_3_test.drop(df_user_ids_3_validate.index)
df_user_ids_4_train = df_user_ids_4.sample(frac=0.8, random_state=0)
df_user_ids_4_test = df_user_ids_4.drop(df_user_ids_4_train.index)
df_user_ids_4_validate = df_user_ids_4_test.sample(frac=0.5, random_state=0)
df_user_ids_4_test = df_user_ids_4_test.drop(df_user_ids_4_validate.index)

In [41]:
# combine all dataframes having train in one them   
df_user_ids_train = pd.concat([df_user_ids_0_train, df_user_ids_1_train, df_user_ids_2_train, df_user_ids_3_train, df_user_ids_4_train])
df_user_ids_test = pd.concat([df_user_ids_0_test, df_user_ids_1_test, df_user_ids_2_test, df_user_ids_3_test, df_user_ids_4_test])
df_user_ids_validate = pd.concat([df_user_ids_0_validate, df_user_ids_1_validate, df_user_ids_2_validate, df_user_ids_3_validate, df_user_ids_4_validate])

In [42]:
df_user_ids_train.shape, df_user_ids_test.shape, df_user_ids_validate.shape

((121, 2), (15, 2), (15, 2))

In [46]:
# create new dataframe from df with only train user ids get all tweets for each user id
df_train = df[df['twitter user id'].isin(df_user_ids_train['twitter user id'])]
df_test = df[df['twitter user id'].isin(df_user_ids_test['twitter user id'])]
df_validate = df[df['twitter user id'].isin(df_user_ids_validate['twitter user id'])]

In [47]:
df_train.shape, df_test.shape, df_validate.shape

((649, 4), (66, 4), (76, 4))

In [49]:
df_train.to_csv('../data/train.csv', index=False)
df_test.to_csv('../data/test.csv', index=False)
df_validate.to_csv('../data/validate.csv', index=False)

In [23]:
df_nano = df[df['class'] == 'nano']
df_macro = df[df['class'] == 'macro']
df_mega = df[df['class'] == 'mega']
df_no_influencer = df[df['class'] == 'no influencer']
df_micro = df[df['class'] == 'micro']

print(df_nano.shape, df_macro.shape, df_mega.shape, df_no_influencer.shape, df_micro.shape)

(158, 4) (177, 4) (213, 4) (74, 4) (169, 4)


In [24]:
df_nano_grp = df_nano.groupby('twitter user id').agg({'texts': ' '.join})
df_macro_grp = df_macro.groupby('twitter user id').agg({'texts': ' '.join})
df_micro_grp = df_micro.groupby('twitter user id').agg({'texts': ' '.join})
df_mega_grp = df_mega.groupby('twitter user id').agg({'texts': ' '.join})
df_no_influencer_grp = df_no_influencer.groupby('twitter user id').agg({'texts': ' '.join})

In [25]:
df_nano_grp['text_length'] = df_nano_grp['texts'].apply(lambda x: len(x))
df_macro_grp['text_length'] = df_macro_grp['texts'].apply(lambda x: len(x))
df_micro_grp['text_length'] = df_micro_grp['texts'].apply(lambda x: len(x))
df_mega_grp['text_length'] = df_mega_grp['texts'].apply(lambda x: len(x))
df_no_influencer_grp['text_length'] = df_no_influencer_grp['texts'].apply(lambda x: len(x))

In [8]:
def get_df_with_cluster(df, num_topics):
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)
    tf = tf_vectorizer.fit_transform(df['texts'])
    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=25, learning_method='batch', 
                                random_state=42, evaluate_every= 0.5)
    lda.fit(tf)
    doc_topic_weights = lda.transform(tf)
    scaler = StandardScaler()
    nano_scaled = scaler.fit_transform(doc_topic_weights)
    kmeans = KMeans(n_clusters=5)
    kmeans.fit(nano_scaled)
    cluster_labels = kmeans.predict(nano_scaled)
    df['cluster'] = cluster_labels
    return df

In [9]:
df_nano_return = get_df_with_cluster(df_nano_grp, num_lda_topics)

In [14]:
df_nano_return

texts  \
cluster twitter user id                                                                       
0       aaa1584ceaad35c0a52a42803965c189  rt braver crypto bear market giveaway back fir...   
        aeb262e188dfd453e16d7b5909874968  rt wl giveaway prizes 2 x allowlist spots ente...   
        5bb03559fe50a06839c1a7aef1c94ceb  rt going make time special banksy promise tsla...   
        f73ecbc0d4d8fc84718f80a3e2cb7d59  rt ready lion flying_saucer mint september eth...   
1       f77549dbdcbf38126210e5d5a9db98eb  rt todays ga celebrate women ens amazing see g...   
        ef3ebcf58e6468ea3c2fb3ad79540ad4  rt receive 86 000 000 86 000 000 shib first 30...   
        3e506c96800b45814613cc46a241385a  rt one first gacc got 2 months ago stoned dmt ...   
        9874e41d54a664d5bb5ac76346903ac7  rt iota launched smart contract beta makes eth...   
        f8404b995e68fac31ac3f8318884a0a9  rt excited announce last round 1inch airdrop s...   
        e3b4e60374658a005fa49fdbad1d516c  rt asia continues board 1inch plugs klaytn ent...   
2       d2335a22712cde118b103869bfaa594f  rt wants tether drop usdt address follow amp r...   
        1bc3271b139a2c06af59e579890796f7  join waggle network public sale whitelist use ...   
        c88b607601949a0b47baa93f2e0693c5  rt solanium giveaway thread thread 1 celebrate...   
        3ddbf5b440130d3ee61d4ac28018b498  rt earnings week tsla aapl msft ba intc jnj ha...   
        fec182516cba4b665e2215094bbcc527  rt eyes cup_with_straw iykyk ps reach 300 foll...   
        4297a11cc420930468d7b5db3cbddf82  rt hits 10 million tvl 96hrs launch rocket roc...   
        062492818c984febba843b650a4a602e  takes 2 matic 100 bank polygon one lowest cost...   
        1bc1201be24ca9d0c0afd94ca0116609  nice lkmex rewards coming people minted elrond...   
3       67a78a892dda5165aa35f6d2cc55f2e7  rt millennialcoin airdrop campaign date august...   
        11727505346c5f966891f74094042073  rt amount time ape halts 1 18 sec 2 30 sec 3 2...   
4       729eebc9e15bf95dd30b55d894c9799a  rt ultra giveaway 1x 1 sol declared minting da...   
        03eaa72711143b521c073d9ac5745923  rt need knock heels together three times say s...   
        a847260a6a065136ef60c95e02ee1996  rt wrapped_gift giveaway wrapped_gift prizes t...   
        a3259c5facf41f2e842309743486f1cc  best thing guess reason preferred stock div cr...   
        0f0942696ae8bcadf0db494cce7333e0  rt receive 870 000 000 00 870 000 000 00 shib ...   
        e5e9999715d502de2153189601f0a73d  rt btc amp bnb dramatic falls last 24 hours hi...   
        3686962059845fca97dd56fee083e65f  possibility market makers pulling liquidity am...   
        c543d9335fa9289048d81af27371848e  rt 1 thread currently works fud wrong real pro...   
        0037a672f0ed64b3231bac64853a278d  rt ape ape give apes tweeting great pics atamc...   
        89175626a312db4e3988f714b1497ac7  rt every day miners selling around 20 million ...   
        dd5f93bbd854c3f7c88abf61476c66bf  rt enjoying flexin artist 1 supporter thinking...   
        e90bd50625748dcd6f9d40d932ab2231  rt sol hits 200 today give everyone rt likes 4...   

                                          text_length  cluster  
cluster twitter user id                                         
0       aaa1584ceaad35c0a52a42803965c189          986        0  
        aeb262e188dfd453e16d7b5909874968          804        0  
        5bb03559fe50a06839c1a7aef1c94ceb          168        0  
        f73ecbc0d4d8fc84718f80a3e2cb7d59           91        0  
1       f77549dbdcbf38126210e5d5a9db98eb          789        1  
        ef3ebcf58e6468ea3c2fb3ad79540ad4          775        1  
        3e506c96800b45814613cc46a241385a          739        1  
        9874e41d54a664d5bb5ac76346903ac7          652        1  
        f8404b995e68fac31ac3f8318884a0a9           88        1  
        e3b4e60374658a005fa49fdbad1d516c           53        1  
2       d2335a22712cde118b103869bfaa594f         1444        2  
        1b

In [ ]:
# group dataframe by cluster and sort each cluster by text_length column
df_nano_return = df_nano_return.groupby('cluster').apply(lambda x: x.sort_values(['text_length'], ascending=False))

In [15]:
df_macro_return = get_df_with_cluster(df_macro_grp, num_lda_topics)

In [16]:
df_macro_return = df_macro_return.groupby('cluster').apply(lambda x: x.sort_values(['text_length'], ascending=False))

In [17]:
df_macro_return

texts  \
cluster twitter user id                                                                       
0       383e21ef21cf28f9f3005c9fc6903109  peak bull like top 50 coins possible disrupt a...   
        d86fbb678a84112f8914c63e46531d21  locking period double rewards low transactions...   
        eb609388df96de828143f2830c89251a  anticipated earnings july 25th july 29th check...   
        c0d3cb706dd6ad09cb7e0b7858f98e63  good morning cole everyone happy labor day wee...   
        5be065e13b80dad05212f137e83deb38  winner trophy congratulations party_popper con...   
        991870dafafeceb067d2ef608daed64b  rt hon butiime order give force compensation s...   
1       aec6c5d7bde4579b1af1e6f3dd067c57  rt bitmart monthly report december 2021 dizzy ...   
        84993708d20e2dffb5807b6b510624f0  anatol lieven anyone expresses admiration anym...   
        96e9d592dfc66654b653cb2822be1760  tiktok growing platform new creators get invol...   
2       a6623aea011de0796dab71f92c921e7d  huge divergence small caps iwm qqq think fed m...   
        2a4594727182fe436cf4c92ff95afd5f  truss got lot better real debate sunak belter ...   
        1fe087134fe681f8509f8d9ced432621  lol yeah everything wrong world lololol think ...   
        892329d79b72cbf55b8533ca71b28326  got ta get bowl man even enjoy days think putt...   
3       bbfa98f4c0bae570084ebf9e441cbb17  uba crypto traders female nft launched alarm_c...   
        f49dfd7536d3f532ec45d4663e6f421b  lem predict next like actually deposited eth a...   
        cfa00327235235fe3cc72fc5449ee111  choose category person waits feel like somethi...   
        1316cba6e1442a126102d5e2ab0813ec  rt man describing boobahub innit 1 2b fdv 130m...   
4       38db4eddf03117ce91093a920966451e  digital currency gains presence india indian f...   
        eed932b247e47a2c48dae5d8cc74dc4e  rt special moment without special oat right st...   
        0d3700fa5c7c3fce6fd1e1ffd5282f50  data shows bitcoin could verge bull run crypto...   
        13e873428de3ccfd3873b3d7258ad411  members management content portal marketplace ...   
        794fa4ae5ee04ffc401c1981be40a5b5  rt 2022 canada officially arrived world stage ...   
        3eaa1b5e4afa67fb2d097da6aae9d59d  rt look egld mooning smiling_face_with_heart e...   
        fa8d3942f7e1420a5a3d9d19b672f8f2  holy sh web3 husbando thing thank ffluxurywatc...   
        92eb6f1f295ca6ad74e3aee02b9f144d  rt 20 60 minutes join tg rt follow people defe...   
        2f5a43bbfb7fcb3623222de6305b45b9  rt mike amp dustin always sticking together ga...   
        5509d8b2ba81eb6bc580a8aaa89b335a  microsoft research multitouch mice favorite di...   
        4afb67b167a44d101a5b6ef2d2272d49  congrats man fire fire fire put use mouse scro...   

                                          text_length  cluster  
cluster twitter user id                                         
0       383e21ef21cf28f9f3005c9fc6903109         1801        0  
        d86fbb678a84112f8914c63e46531d21         1253        0  
        eb609388df96de828143f2830c89251a          906        0  
        c0d3cb706dd6ad09cb7e0b7858f98e63          671        0  
        5be065e13b80dad05212f137e83deb38          657        0  
        991870dafafeceb067d2ef608daed64b           58        0  
1       aec6c5d7bde4579b1af1e6f3dd067c57         1380        1  
        84993708d20e2dffb5807b6b510624f0         1193        1  
        96e9d592dfc66654b653cb2822be1760          936        1  
2       a6623aea011de0796dab71f92c921e7d          815        2  
        2a4594727182fe436cf4c92ff95afd5f          726        2  
        1fe087134fe681f8509f8d9ced432621          647        2  
        892329d79b72cbf55b8533ca71b28326          195        2  
3       bbfa98f4c0bae570084ebf9e441cbb17         1351        3  
        f49dfd7536d3f532ec45d4663e6f421b          667        3  
        cfa00327235235fe3cc72fc5449ee111          474        3  
        1316cba6e1442a126102d5e2ab0813ec          245        3  


In [18]:
df_micro_return = get_df_with_cluster(df_micro_grp, num_lda_topics)
df_micro_return = df_micro_return.groupby('cluster').apply(lambda x: x.sort_values(['text_length'], ascending=False))
df_micro_return

texts  \
cluster twitter user id                                                                       
0       787ad6a075042caeb746621efdbdcd85  halo web3 free play today want earn sol playin...   
        f3b13776eb061e4754123e31432f5b68  leggo lt 3 sol investment advice group busines...   
        31372df13f6fc329b520357ad3cde349  manage risk lose either way degen ape mentalit...   
1       b8a86d5eaebddded3feaf7812f07fe0c  realizing thing defi blue chip industry young ...   
        edbf78f8f8abfd87f8cbe08023ae0eaf  checked floor one close minted 2 75 eth 3 160 ...   
        be9244da7b1a2b634d443aca3288a633  open secret almost nft marketplaces soon suppo...   
        8c43bc7498c69a3678cf7b4853014351  tvl 2 91 billion usd market cap 12 billion con...   
        fb41227b22d727fbcff4fe780d849de6  apes got bagged bbby amc ape guess lol trader ...   
        e220b06b623b0b1a406e00e3167e5566  rt pow price go eth needs 600m per month new c...   
        d4ff198244a9671c78c4105f93085998  hmmm buy ape week amc ape units meme friendly ...   
        c727fd8b7dc3978b363de67dad8c40af  rt profits go support sibling heart failure fl...   
2       60f11142017b254d7c2cc7ad60667732  hey frens wrong zany_face platform save crypto...   
        44717bda8fff19613beb860dcda3b4f3  rt icp mood one gif mine ic friends broom broo...   
        4a5ac93c64b3c8eb9c6467db2f339150  rt speaker_high_volume speaker_high_volume goo...   
        e65d54d66c3290b61636e55ef180b5c2  1 eth sol winner rt like amp follow turn notis...   
        f1f4cd93cf28218caa00ffd687907510  rt ready sign online person hackathon starting...   
        dd06435f7c24ad2df8fafecb951c2218  rt kpmg last 100 tweets 2 likes average announ...   
        91d718ddeb59cfd16fb7058758a94d87  rt heard friend crypto tax accountant client a...   
        0409fe210a0edfe258d21e3404e1ce05  rt top 10 solana collection owners opensea rt ...   
        592760b24ca4057a13c771b1aa7275d6  anyone know pre attack luna holders getting ai...   
        2e30287cca6193247154f34ba1127ecc  rt watching drama drawing conclusion bullish c...   
        a6a416360fb28553d0a7ffb88467e6ee  police_car_light new integration alert automat...   
        a8d8d5d1241fd71278f152cd24ac8323  loudspeaker elrondbet raffle alarm_clock 8 hou...   
        40f4c95f8453f0db952cc7bf65c69c1d  rt matic expecting new candle close demand zon...   
        28ab93d1cfa1452bee4da4a93746b956  rt ada trend line trend line 2 breaks market l...   
        5539bcf90fdeb5f72914037f51147ba2  rt breaking gme possibly secretly bought lrc e...   
        ecb600dcc20d30353391b9ed293caf8a  rt wrapped_gift sol amp whitelistgiveaway wrap...   
        7b40160e66771ab4762fed50d24ac35d  rt 2000 usdt 2 winners money_with_wings rocket...   
3       5ac42dd7ef9de9c1ac5d80dfa17ed552  comment team check_mark_button team a_button_ ...   
        96cd6178dd1341cd892b7c13d5158e2b  winner twitter usernames send dm twitter profi...   
4       78bd3bf2d4245f31ca82e4b748eb8847  rt top parachains number holders polkadot let ...   

                                          text_length  cluster  
cluster twitter user id                                         
0       787ad6a075042caeb746621efdbdcd85          825        0  
        f3b13776eb061e4754123e31432f5b68          636        0  
        31372df13f6fc329b520357ad3cde349          162        0  
1       b8a86d5eaebddded3feaf7812f07fe0c         1379        1  
        edbf78f8f8abfd87f8cbe08023ae0eaf         1277        1  
        be9244da7b1a2b634d443aca3288a633         1050        1  
        8c43bc7498c69a3678cf7b4853014351          827        1  
        fb41227b22d727fbcff4fe780d849de6          580        1  
        e220b06b623b0b1a406e00e3167e5566          171        1  
        d4ff198244a9671c78c4105f93085998          122        1  
        c727fd8b7dc3978b363de67dad8c40af           71        1  
2       60f11142017b254d7c2cc7ad60667732         1710        2  
        44717bda8fff19613beb860dcda3b4f3

In [19]:
df_mega_return = get_df_with_cluster(df_mega_grp, num_lda_topics)
df_mega_return = df_mega_return.groupby('cluster').apply(lambda x: x.sort_values(['text_length'], ascending=False))
df_mega_return

texts  \
cluster twitter user id                                                                       
0       d8816b8d2b001ffe1bf47c5e3b1d8a9e  alarm_clock hurry starting 30 minutes play_but...   
        1cbf16d93cc6a672eaed9c16ac07f114  fine got amazing shoot dm rocket envelope_with...   
1       f4bd8c36c48b0e7e12b37733d7d98dd6  rt dear polygon family dizzy something celebra...   
        2e9a06568e70652d1ccedff0b433b43e  rt police_car_light 10 000 giveaway money_with...   
        a895f3e34691f388ee48b833570e3690  13 step keycap_5 luna airdrop sent vested new ...   
        623113eb843163bf06c9e75d7ca56961  29 year old built career 2 side hustles earn 1...   
        a2f7726319df31c397e4ad794d9da217  national cancer survivors day help us celebrat...   
        5a3336a8e007d3848b30a8cab701f684  rt canada formally labels proud boys terrorist...   
        d3202f7bed1bc52575b315a8c55b68b6  good job drop dm guys speech_balloon fam fire ...   
        da3c028d3c264701fef2eded8281ec2f  niggaz eating swag wan na admit rt happy birth...   
        38ed8d57396dd002f1fc2ba6db91081b  amc theatres considering adding shiba inu shib...   
        c5ca7fe528438feb3a389d90d88c284d  muh cheap fees narrative working roger rt unde...   
        21658f2f073f0e041b79bf0f7632a0eb  rt check new single sacrifice fire philip gard...   
2       e0220feef7cfdc682f6e2b1e089087bf  web3 land sale small_orange_diamond 550 lands ...   
        72dd8dd2f83ba3e8ba2c613653379605  think bitcoin remain standard course stay alon...   
        5ac824229805900a92e53e4867e54f49  rt facts tho lex changed whole sound rap hundr...   
        4bdb0a8856f8109bc2fca7c188aa8986  happppppiest birthdays time favorite hero love...   
        179aa6d5d94999ae0bf77d13bcc5fa31  feels good right money mouth_face blows mind p...   
        c11cc2cb8e46e082c134ad7cbfe18398  check time zones friend smiling_face_with_sung...   
        cc7a7d39a686a70461b54d402f06d457  connect wallet mint torches genesis nfts party...   
        750aba76c971350810ebe369f8ae0b7a            money_bag awards credited within 1 week   
3       3fd08651cbffb8d0835e47c819ded8d5  slow return cinema going covid 19 pandemic fil...   
        4c2f3c58ec4996fe74ebbfadffdd5428  energy crisis resulted exodus bitcoin miners k...   
        318e755c806d7edc16b2fb7518e3fbde  rt julian assange brother speaks ahead expecte...   
        4436cef2cd8b7fd00e72ed2592694da2  less 5 days left register clapping_hands prize...   
        69dbb1f2410ee483faefbee04a13039c  hey cancun seeya dec 16 17 18 friends tix sale...   
4       dd6d04b11ed5c18f601cdbd0a467289a  rt paymeinbitcoin got get paid bitcoin love ap...   
        a9a3cb55feb90e9d4fd223a1683e3ca8  rt love latest group shot shirt rt went first ...   
        d099b9f432709659c04523ffdf0ac200  good sleep afternoon way stadium good morning ...   
        f4d8a2a01c79ba39feed9ec8b2f9acd8  hello sorry hear may please send us details pr...   

                                          text_length  cluster  
cluster twitter user id                                         
0       d8816b8d2b001ffe1bf47c5e3b1d8a9e         1043        0  
        1cbf16d93cc6a672eaed9c16ac07f114          331        0  
1       f4bd8c36c48b0e7e12b37733d7d98dd6         1231        1  
        2e9a06568e70652d1ccedff0b433b43e         1026        1  
        a895f3e34691f388ee48b833570e3690          906        1  
        623113eb843163bf06c9e75d7ca56961          876        1  
        a2f7726319df31c397e4ad794d9da217          704        1  
        5a3336a8e007d3848b30a8cab701f684          632        1  
        d3202f7bed1bc52575b315a8c55b68b6          522        1  
        da3c028d3c264701fef2eded8281ec2f          518        1  
        38ed8d57396dd002f1fc2ba6db91081b          466        1  
        c5ca7fe528438feb3a389d90d88c284d          440        1  
        21658f2f073f0e041b79bf0f7632a0eb          364        1  
2       e0220feef7cfdc682f6e2b1e089087bf         1409        2  
     

In [26]:
df_no_influencer_return = get_df_with_cluster(df_no_influencer_grp, num_lda_topics)
df_no_influencer_return = df_no_influencer_return.groupby('cluster').apply(lambda x: x.sort_values(['text_length'], ascending=False))
df_no_influencer_return

texts  \
cluster twitter user id                                                                       
0       d75542f32c8d3c4cd3533f7c6ad1916f  cloud_with_lightning giveaways cloud_with_ligh...   
        7b6f2b51b08558e7aae334a806920107  rt police_car_light giveaway alert police_car_...   
        8ba8b37c646a965c8de4ffcd7598572c  wait egld conference 3rd november count rocket...   
1       be558c5b8dd1a7b9b1ba05fbc324d693  great project revolutionarize cryptocurrency i...   
        95b6f482041b5d683321aa3761f28c9d  best project 3rcqxzrzlshwhk5wblamutrga61suufvn...   
        d5dd10cd702a791660eb98f7a1a6871f  fire heart_on_fire fire fire heart_on_fire sun...   
        bece0f146543ad67467f7a16bb78e707  nice project smiling_face_with_heart eyes 3ytf...   
        e1ec91e2c7f3fab9f6d5b10039b58b19  picking 2 eth winners like rt tag 3 frens back...   
        f5e0db8da605dc466e4bebf0333938a0  introducing scrappyether double_exclamation_ma...   
        0ed9637249db91cb2c256ec156ce1977  updated price qtum bnb trx qtum 3 361 usd bnb ...   
        7f126542cda1a312b08f1b2ea82f3642  cool innovative project price 0 1 sol supply 4...   
        e33230fd7719dd3ba5df50869fdd42e6  rt couch potato mint couchpotatoes bsc bnb eth...   
        0e1d2c43b93e8e80dc8eb6b29d48b2c1  sending bunch eth random people post notificat...   
2       36b3a60bab1790a46d9f54284560dc8b  exposes badly want us sell ape diamond hand ap...   
        b73ff380c0ddba81d98a0240613d6023  sorry seen enough delusion hold tongue amc ape...   
        33d2e20cd6ce5b395bec12f7d0853816  money_bag wan na win hard cash partying_face w...   
        a499049f5d596b75bd1a036602216bc4  luna classic going go crazy lfg rename page po...   
        5a62bef88fac1e29c3d0fb9be080e8f7  spouting_whale cardano ada whale red_heart lac...   
        6b9d302f2f18ed5eacbcc59f53a2fa2b  defeated luna quest record time played yet nft...   
        da89c2c020fcec3484dc7c59e84c85f6  one thing learn projects like xrp ada maintain...   
        5d516c64beebffd632b1698cd3b94efb  อย ด ๆก ได luna แพงๆมาเต มม อ face_holding_bac...   
        5d09f92f130550e5b01508c930de16bc  live cointigerex cointiger quack usdt start tr...   
3       af71fc2bd031be154a3334ac4ce5b0a2  next bezoge run gon na wild mark words market ...   
        faed79f6d4a62e1c984a62d064f2c8d6  buy babyfd babyflokidoge one month old bsc 1m ...   
        d93c34891319f552eec1afed5241bb62  last days 1b 250m worth btc injected ada binan...   
        68e43b6c5bcad14eb7e555bdde629f1c  rt top 10 coins mentioned twitter last week me...   
        bc2283776d5428ebfe65f1a3574e3ca1  first crypto project movetoearn auto staking a...   
4       6bf325565eeff0d73d4c1c31aabfcba9  requesting matic funds stakely faucet polygon ...   
        9a6edc182760ef98796afad9783341dd  backhand_index_pointing_right joined ido white...   
        66f29c09663328ecf96a8dc28e02ed72  requesting sol funds stakely faucet solana blo...   

                                          text_length  cluster  
cluster twitter user id                                         
0       d75542f32c8d3c4cd3533f7c6ad1916f          458        0  
        7b6f2b51b08558e7aae334a806920107          146        0  
        8ba8b37c646a965c8de4ffcd7598572c          118        0  
1       be558c5b8dd1a7b9b1ba05fbc324d693         1134        1  
        95b6f482041b5d683321aa3761f28c9d          351        1  
        d5dd10cd702a791660eb98f7a1a6871f          195        1  
        bece0f146543ad67467f7a16bb78e707          125        1  
        e1ec91e2c7f3fab9f6d5b10039b58b19          107        1  
        f5e0db8da605dc466e4bebf0333938a0          107        1  
        0ed9637249db91cb2c256ec156ce1977           96        1  
        7f126542cda1a312b08f1b2ea82f3642           73        1  
        e33230fd7719dd3ba5df50869fdd42e6           71        1  
        0e1d2c43b93e8e80dc8eb6b29d48b2c1           69        1  
2       36b3a60bab1790a46d9f54284560dc8b          514        2  
     